In [111]:
import numpy as np
from potentials import BuckinghamPotential
from potential_parameters import BuckinghamParameter
from atom_types import AtomType
from bond_types import BondType
from bonds import Bond
from atoms import Atom
from pymatgen.io.vasp.outputs import Vasprun
from lammps_data import LammpsData
from unittest import mock
from mock import Mock, patch, call, mock_open
import pytest

In [93]:
vasprun = Vasprun('test_files/test_vasprun.xml')
structure = vasprun.ionic_steps[0]['structure']
structure.add_site_property('forces', np.array(vasprun.ionic_steps[0]['forces']))

/Users/lmm82/.pyenv/versions/3.7.3/lib/python3.7/site-packages/pymatgen/io/vasp/outputs.py:907: UserWarning: No POTCAR file with matching TITEL fields was found in /Users/lmm82/Documents/current_work/lammps_potenial_fitting/test_files
  " was found in {}".format(os.path.abspath(p)))


In [94]:
params = {}
params['core_shell'] = { 'Li': False, 'Ni': False, 'O': True }
params['charges'] = {'Li': +1.0,
                     'Ni': +3.0,
                     'O': {'core':  -2.0,
                           'shell': 0.0}} 
params['masses'] = {'Li': 6.941,
                    'Ni': 58.6934,
                    'O': {'core': 14.3991,
                          'shell': 1.5999} }
params['potentials'] = {'Li-O': [100.00, 0.1, 0.0],
                        'Ni-O': [1000.00, 0.2, 0.0],
                        'O-O':  [10000.00, 0.3, 0.0]}
params['cs_springs'] = {'O-O' : [10.0, 0.0]}

In [98]:
def collate_structural_data(params, structs, supercell=None):        
    vaspruns = ['test_files/vasprun_small.xml' for i in structs]
    lammps_data = [data_from_vasprun(params, v, i, supercell) for i, v in enumerate(vaspruns)]
    return lammps_data


def data_from_vasprun(params, filename, i, supercell):
    vasprun = Vasprun(filename)
    structure = vasprun.ionic_steps[0]['structure']
    structure.add_site_property('forces', np.array(vasprun.ionic_steps[0]['forces']))
    stresses = vasprun.ionic_steps[0]['stress']
    if supercell is not None:
        if isinstance(supercell[0], int) and len(supercell) == 3:
            structure = structure*supercell
        elif isinstance(supercell[i], list) and (all([len(supercell[i]) == 3 for i, x in enumerate(supercell)])):
            structure = structure*supercell[i]
        else:
            raise ValueError('Incorrect dimensions for supercell. Requires x,y,z expansion (i.e. list of 3 integers) or list of x,y,z expansions for each structure (i.e. list of list(x,y,z))')

    struct_data = LammpsData.from_structure(structure, params, i, stresses)
    return struct_data

In [99]:
@pytest.fixture
def params():
    params = {}
    params['core_shell'] = { 'Li': False, 'Ni': False, 'O': True }
    params['charges'] = {'Li': +1.0,
                         'Ni': +3.0,
                         'O': {'core':  -2.0,
                               'shell': 0.0}} 
    params['masses'] = {'Li': 6.941,
                        'Ni': 58.6934,
                        'O': {'core': 14.3991,
                              'shell': 1.5999} }
    params['cs_springs'] = {'O-O' : [10.0, 0.0]}
    return params

@pytest.fixture
def lammps_data():
    lammps_data = mock.Mock(LammpsData)
    lammps_data.file_name = 'test_files/vasprun_small.xml'
    return lammps_data

@mock.patch('collate_structures.data_from_vasprun', return_value=lammps_data, autospec=True)
def test_output_in_collate_structural_data(mock_data_from_vasprun, params):
    output = collate_structural_data(params, [0,1])
    assert len(output) == 2
    assert type(output) == list
#     assert len(output) == 2
#     assert output[0].file_name == 'test_files/vasprun_small.xml'

In [100]:
output = test_output_in_collate_structural_data(params)

AttributeError: 'function' object has no attribute 'keys'

In [106]:
np.zeros((10,3))

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [107]:
from scipy import optimize

In [109]:
fit_output = mock.Mock(spec=optimize.OptimizeResult)